# ML Lab Assignement 02
***Question***: A banking institution wants to develop a model that predicts whether a customer will subscribe to a term deposit or not based on their banking behaviour. They have collected a dataset of past customers with information about their demographics, account details, and whether they subscribed to a term deposit or not. The bank wants to build a model using Logistic Regression to predict whether a new customer will subscribe to a term deposit or not based on their banking behaviour.The dataset used for this problem statement is the "Bank Marketing Data Set" which contains information on customers of a bank. The dataset includes 17 attributes such as customer demographics, account details, and whether the customer subscribed to a term deposit or not. Each customer is classified as either yes or no based on whether they subscribed to a term deposit or not.

## Import the libraries

In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from collections import Counter

## Import The Dataset

In [3]:
df = pd.read_csv('/kaggle/input/bank-dataset/bank-full.csv',sep=';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [5]:
df.describe().astype(np.int64)

,age,balance,day,duration,campaign,pdays,previous
count,45211,45211,45211,45211,45211,45211,45211
mean,40,1362,15,258,2,40,0
std,10,3044,8,257,3,100,2
min,18,-8019,1,0,1,-1,0
25%,33,72,8,103,1,-1,0
50%,39,448,16,180,2,-1,0
75%,48,1428,21,319,3,-1,0
max,95,102127,31,4918,63,871,275


# Logistic Regression

## spliting data 80:20 , 80% for training 20% for testing

### Utilize scikit-learn's train_test_split function to divide the data into:
* Training set (80% of the data): Used to train the model.
* Testing set (20% of the data): Used to evaluate the model's performance on unseen data. 


In [6]:
X = df.drop(columns=['y'])
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)

Number transactions X_train dataset:  (36168, 16)
Number transactions y_train dataset:  (36168,)
Number transactions X_test dataset:  (9043, 16)
Number transactions y_test dataset:  (9043,)


In [7]:
X_test

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
14001,37,management,married,tertiary,no,1403,yes,no,cellular,10,jul,91,15,-1,0,unknown
32046,36,services,single,secondary,no,-253,yes,no,cellular,14,apr,1073,1,-1,0,unknown
13318,47,blue-collar,married,primary,no,-406,no,yes,telephone,8,jul,243,5,-1,0,unknown
42991,57,housemaid,married,primary,no,501,yes,yes,cellular,11,feb,105,7,188,2,failure
14237,33,blue-collar,married,secondary,yes,-406,yes,yes,cellular,14,jul,332,2,-1,0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18448,53,blue-collar,married,primary,no,252,no,no,cellular,31,jul,727,4,-1,0,unknown
6654,28,management,single,tertiary,no,1996,yes,yes,unknown,28,may,55,1,-1,0,unknown
17106,28,services,married,secondary,no,7,no,no,cellular,25,jul,50,3,-1,0,unknown
9025,59,retired,married,secondary,no,0,no,no,unknown,5,jun,812,1,-1,0,unknown


## Handle the categorical variables
## clasify it to numerical and categorical feature
Understand the different data types present in our features, which is crucial for feature engineering and model selection.This separation allows us to apply appropriate techniques to each type. We can explore numerical features using descriptive statistics and visualizations, while categorical features might benefit from frequency tables and bar charts.

In [8]:
numeric_features = X_train.select_dtypes(include=['float64', 'int64']).columns.values
numeric_features = numeric_features[numeric_features != 'y']

category_features = X_train.select_dtypes(include=['object', 'bool']).columns.values

print("numeric features",numeric_features)
print("categorical features",category_features)

numeric features ['age' 'balance' 'day' 'duration' 'campaign' 'pdays' 'previous']
categorical features ['job' 'marital' 'education' 'default' 'housing' 'loan' 'contact' 'month'
 'poutcome']


In [9]:
def dummify(ohe, x, columns):
    transformed_array = ohe.transform(x)

    enc = ohe.named_transformers_['cat'].named_steps['onehot']
    feature_lst = enc.get_feature_names_out(category_features.tolist())   
    
    cat_colnames = np.concatenate([feature_lst]).tolist()
    all_colnames = numeric_features.tolist() + cat_colnames 
    
    df = pd.DataFrame(transformed_array, index = x.index, columns = all_colnames)
    
    return transformed_array, df

In [10]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, category_features)])

ohe = preprocessor.fit(X_train)

X_train_t = ohe.transform(X_train)
X_test_t = ohe.transform(X_test)

In [11]:
X_train_t_array, X_train_t = dummify(ohe, X_train, category_features)
X_test_t_array, X_test_t = dummify(ohe, X_test, category_features)

X_train_t.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
13932,1.515061,-0.231637,-0.698514,1.808943,-0.567836,-0.410222,-0.241405,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9894,-0.369597,0.106038,-0.818717,-0.755580,-0.567836,-0.410222,-0.241405,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
39946,-0.558063,0.470635,-1.660136,-0.604270,0.402837,0.560728,0.585721,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9217,-0.558063,-0.375660,-1.299528,-0.041704,-0.567836,-0.410222,-0.241405,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4124,-0.275365,-0.340303,0.383311,-0.464598,-0.567836,-0.410222,-0.241405,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
X_train_columns = X_train_t.columns
print(X_train_columns)

Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'education_primary', 'education_secondary', 'education_tertiary',
       'education_unknown', 'default_no', 'default_yes', 'housing_no',
       'housing_yes', 'loan_no', 'loan_yes', 'contact_cellular',
       'contact_telephone', 'contact_unknown', 'month_apr', 'month_aug',
       'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'poutcome_failure', 'poutcome_other', 'poutcome_success',
       'poutcome_unknown'],
      dtype='object')


## Oversampling
We are going o use somte technique for class imbalance,counting the occurances of each class in the training target variable to understand the imbalance
Create a smote object to use it for:
* Fit the model to the training data.
* Generate synthetic samples for the minority classes.
* Oversample the dataset to create a more balanced distribution.

In [13]:
from imblearn.over_sampling import SMOTE

counter = Counter(y_train)
print(counter)

oversample = SMOTE()
X_train_smote, y_train = oversample.fit_resample(X_train_t, y_train)

counter = Counter(y_train)
print(counter)

Counter({'no': 31942, 'yes': 4226})
Counter({'yes': 31942, 'no': 31942})


## Random Feature Elimination
* Here we're going to prepare the data for feature selection.
* Afterthat we will apply random feature selection initializing an RFE object using a Logistic Regression base model with specific solver and iteration settings.
* Then we will prepare our final data for testing

In [14]:
from sklearn.svm import SVC


final_X_train = pd.DataFrame(data=X_train_smote,columns=X_train_columns )
final_y_train = pd.DataFrame(data=y_train,columns=['y'])

rfe_model = RFE(LogisticRegression(solver='lbfgs', max_iter=1000),n_features_to_select=25)
rfe_model = rfe_model.fit(final_X_train, final_y_train)

# feature selection
print(rfe_model.support_)
print(rfe_model.ranking_)

[False False False  True False False False False False  True  True False
  True False False  True False False  True False False False  True False
 False False  True False  True  True False  True False False  True False
  True  True False  True  True False  True  True  True  True  True  True
  True  True  True]
[27 23 25  1  5 24 22  9 14  1  1 19  1  8 15  1 20 16  1 18 17  7  1 21
 10 12  1 11  1  1  3  1  2  4  1 26  1  1 13  1  1  6  1  1  1  1  1  1
  1  1  1]


In [15]:
selected_columns = X_train_columns[rfe_model.support_]
print(selected_columns.tolist())

['duration', 'job_entrepreneur', 'job_housemaid', 'job_retired', 'job_student', 'job_unknown', 'education_primary', 'default_no', 'housing_no', 'housing_yes', 'loan_yes', 'contact_unknown', 'month_aug', 'month_dec', 'month_jan', 'month_jul', 'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep', 'poutcome_failure', 'poutcome_other', 'poutcome_success', 'poutcome_unknown']


In [16]:
X_train_final = final_X_train[selected_columns.tolist()]
y_train_final = final_y_train['y']
X_test_final = X_test_t[selected_columns.tolist()]
y_test_final = y_test

X_test_final.head()

,duration,job_entrepreneur,job_housemaid,job_retired,job_student,job_unknown,education_primary,default_no,housing_no,housing_yes,...,month_jul,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
14001,-0.646947,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
32046,3.162980,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
13318,-0.057223,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
42991,-0.592630,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
14237,0.288076,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Model Fitting
### Finally we will crate and train our logistic regression model using LogisticRegression() library
### we will predict the test data and evaluate the accuracy

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

logreg = LogisticRegression()
logreg.fit(X_train_final, y_train_final)

y_pred = logreg.predict(X_test_final)
print('Accuracy of logistic regression classifier on test set: {:.2%}'.format(logreg.score(X_test_final, y_test_final)))

Accuracy of logistic regression classifier on test set: 83.76%


In [18]:
print("Farhan Mahtab Mahi\nReg no. 2018831036")

Farhan Mahtab Mahi
Reg no. 2018831036
